In [1]:
import re

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from pyquery import PyQuery as pq
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(options=chrome_options)
print(driver.title)

Profiles


In [8]:
type(driver)

selenium.webdriver.chrome.webdriver.WebDriver

In [6]:
from bs4 import BeautifulSoup

html=driver.page_source
html = html.replace(u'\xa0', u' ')
res = BeautifulSoup(html, 'html.parser')
soup=res
rows = soup.find_all('div', 'gsc_1usr')
len(rows)

10

In [2]:
def refresh_Pq():
    return pq(driver.page_source)

In [67]:
pi_url_list=[]
l=[]

In [73]:
doc=refresh_Pq()
print(doc('body > div > div.main > div.content.clear > div > div > div.faculty_content span.active').text())
l.append(doc('body > div > div.main > div.content.clear > div > div > div.faculty_content span.active').text())
for i,pi in enumerate(doc.find('body > div > div.main > div.content.clear > div > div > div.faculty_content > div.con_every').items()):
    url=pi('div > a')
    url_text=url.attr('href')
    pi_url_list.append(url_text)
    # print(pi.text())

6


In [75]:
with open('result/xihu_pi_info_url_engineering.txt','a') as f:
    f.write('\n'.join(pi_url_list))

In [76]:
print(len(pi_url_list))

60


## 读取pi personal information

In [3]:
def extract_one_pate_pi_info(pi_link):
    persoanl_page = pq(pi_link, encoding='utf-8')
    chinese_name = persoanl_page(
        'body > div > div.main > div > div.intro.mobile > div.basic_info > p:nth-child(1)').text()
    english_name = persoanl_page(
        'body > div > div.main > div > div.intro.mobile > div.basic_info > p:nth-child(2)').text()[:-7]
    school = persoanl_page('body > div > div.main > div > div.intro.mobile > div.basic_info > p.school')
    school = re.search('school1="(.*?)"', school.text()).group(1)
    website = persoanl_page(
        'body > div > div.main > div > div.intro.mobile > div.basic_info > div > p:nth-child(3) a').text()
    resume = persoanl_page('body > div > div.main > div > div.main_intro > div > div').text()
    return chinese_name,english_name,school,website,resume

In [8]:
from tqdm import tqdm, trange
pi_dic={}
file_list=['result/xihu_pi_info_url_engineering.txt','result/xihu_pi_info_url_science.txt','result/xihu_pi_info_url_SLS.txt',]
for pi_file in file_list:
    print(pi_file)
    with open(pi_file) as f:
        links=f.readlines()
        for pi_link in tqdm(links):
            chinese_name,english_name,school,website,resume=extract_one_pate_pi_info(pi_link.strip())
            pi_dic[chinese_name]=[english_name,school,website,resume]




result/xihu_pi_info_url_engineering.txt


100%|██████████| 60/60 [00:02<00:00, 23.70it/s]


result/xihu_pi_info_url_science.txt


100%|██████████| 55/55 [00:02<00:00, 22.69it/s]


result/xihu_pi_info_url_SLS.txt


100%|██████████| 81/81 [00:03<00:00, 23.16it/s]


In [17]:
westlake_pi_csv_table=pd.DataFrame.from_dict(pi_dic,).T
westlake_pi_csv_table.columns=['english_name','school','website','resume']
westlake_pi_csv_table.index.name='Chinese name'
westlake_pi_csv_table

,english_name,school,website,resume
Chinese name,,,,
陈子博博士,Zibo Chen,生命科学学院与工学院（兼聘）,https://chenlab.org/,个人简介\n\n\n陈子博于2013年在新加坡国立大学获得生命科学一等荣誉学士学位。2013...
程建军博士,Jianjun Cheng,工学院与生命科学学院（兼聘）与理学院（兼聘）,https://www.cheng-lab.com/,个人简介\n\n\n\n程建军教授，1993 年获天津南开大学化学学士学位，1996 年获美...
崔维成博士,Weicheng Cui,工学院,,个人简介\n\n崔维成，1963年生，江苏海门人，“全国优秀科技工作者”（2010）。201...
范迪夏博士,Dixia Fan,工学院,https://www.i4fsi.com/,个人简介\n\n\n范迪夏，1990年出生于上海。2013年获得上海交通大学船舶海洋与建筑工...
郭成辰博士,Chengchen Guo,工学院,https://cguosci.com,个人简介\n\n郭成辰，江苏常州人。2011年6月本科毕业于南京大学化学化工学院，获理学学士...
...,...,...,...,...
郑钜圣博士,"Ju-Sheng Zheng,",生命科学学院,http://zheng.lab.westlake.edu.cn/,个人简介\n\n\n郑钜圣，浙江天台人。2009年毕业于宁波大学，获得学士学位；2014年毕...
周强博士,"Qiang Zhou,",生命科学学院,,个人简介\n周强（1982- ），黑龙江齐齐哈尔人。2000-2004年本科就读于清华大学生...
周挺博士,Ting Zhou,生命科学学院,https://zhouting.lab.westlake.edu.cn,个人简介\n周挺，2006年本科毕业于中国农业大学，2014年博士毕业于中国科学院生物物理研...


## save the pi info

In [18]:
westlake_pi_csv_table.to_csv('result/westlake_pi_csv_table.csv',index=True,encoding='utf8')